In [89]:
import h5py
import numpy as np

ens='cD96'
ensemble='cD211.054.96'

cfg2new=lambda cfg: {'r0':'a','r1':'b','r2':'c','r3':'d'}[cfg[-2:]]+cfg[:4]
cfg2old=lambda cfg: cfg[1:]+'_'+{'a':'r0','b':'r1','c':'r2','d':'r3'}[cfg[0]]

path='/p/project/pines/li47/code/projectData/NST_b-discNJN/{}/data_merge/NST_b.h5_{}'.format(ensemble,ens)
path2='/p/project/pines/li47/code/projectData/NST_b-discNJN-old/data_merge/NST_b.h5_main'
path3='/p/project/pines/li47/code/projectData/NST_b-discNJN/connFromOthers/cD211.054.96/charges_8_20_127srcs.h5'
path4='/p/project/pines/li47/code/projectData/NST_b-discNJN/connFromOthers/cD211.054.96/charges_22_to_26_112srcs.h5'
outfile='dat/NST_b_{}.h5'.format(ens)
with h5py.File(path) as f, h5py.File(outfile,'w') as fw, h5py.File(path3) as f3, h5py.File(path4) as f4:
    cfgs1=[cfg.decode() for cfg in f['cfgs']]
    cfgs3=[cfg2new(cfg) for cfg in f3['gS/up/dt8'].keys()]
    cfgs=list(set(cfgs1).intersection(set(cfgs3)))
    cfgs.sort()
    cfgs_old=[cfg2old(cfg) for cfg in cfgs]
    print(len(cfgs),len(cfgs1),len(cfgs3))
    
    dic={}
    for i,cfg in enumerate(cfgs1):
        dic[cfg]=i
    cfgs1_map=[dic[cfg] for cfg in cfgs]    

    key='VEV/j/data/j+'; fw.create_dataset(key,data=f[key][cfgs1_map])
    key='VEV/j/inserts'; fw.create_dataset(key,data=f[key])
    key='cfgs'; fw.create_dataset(key,data=cfgs)
    key='diags/N/data/N2_N2'; fw.create_dataset(key,data=f[key][cfgs1_map])
    key='diags/N/opabs'; fw.create_dataset(key,data=f[key])

    opabs=[opab.decode() for opab in f['diags/N-j/opabs']]; dic={}
    for i,opab in enumerate(opabs):
        dic[opab]=i
    opabs_new=['g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N','g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
    opabs_map=[dic[opab] for opab in opabs_new]
    
    inserts=[insert.decode() for insert in f['diags/N-j/inserts']]; dic={}
    for i,insert in enumerate(inserts):
        dic[insert]=i
    inserts_new=['id']
    inserts_map=[dic[insert] for insert in inserts_new]
    
    tfs=[int(key.split('_')[-1]) for key in f['diags/N-j/data'].keys()]
    tfs=list(set(tfs))
    tfs.sort()
    key='diags/N-j/opabs'; fw.create_dataset(key,data=opabs_new)
    key='diags/N-j/inserts'; fw.create_dataset(key,data=inserts_new)
    for tf in tfs:
        key='diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)
        t=f[key][:]
        t=t[cfgs1_map]
        t=t[:,:,opabs_map]
        t=t[:,:,:,inserts_map]
        fw.create_dataset(key,data=t)

    for file in [f3,f4]:
        tfs=[int(key[2:]) for key in file['gS/up'].keys()]
        tfs=list(set(tfs))
        tfs.sort()
        for tf in tfs:
            tup=np.array([file['gS/up/dt{}'.format(tf)][cfg][:] for cfg in cfgs_old])
            tdn=np.array([file['gS/up/dt{}'.format(tf)][cfg][:] for cfg in cfgs_old])
            
            key='diags/NJN/data/gS_j+_deltat_{}'.format(tf)
            tP=-(tup+tdn)
            fw.create_dataset(key,data=tP)

            key='diags/N_correlated_NJN/data/dt{}_twop'.format(tf)
            t2pt=np.array([file['dt{}_twop'.format(tf)][cfg][()] for cfg in cfgs_old])
            fw.create_dataset(key,data=t2pt)
    
    # print(cfgs[0])
    # print(cfgs[:6])
    t1=np.real(fw['diags/N/data/N2_N2'][0,:4,0]); t2=np.real(fw['diags/N/data/N2_N2'][0,:4,1])
    print(t1)
    print(t2)
    print((t1+t2)/2)
    print(f3['twop']['0000_r0'][:4])
    print(f4['twop']['0000_r0'][:4])

494 494 496


[2.9934152e-10 6.6069772e-10 4.1016834e-10 2.6603172e-10]
[2.9947020e-10 6.6189043e-10 4.1104325e-10 2.6785130e-10]
[2.9940586e-10 6.6129408e-10 4.1060580e-10 2.6694152e-10]
[4.04645067e-13 6.83991384e-10 4.23079207e-10 2.74955499e-10]
[9.88140125e-13 6.48944475e-10 4.05415645e-10 2.62597853e-10]


In [93]:
import pickle
path='/p/project/pines/li47/code/projectData/NST_b-discNJN/connFromOthers/cD211.054.96/D96_2pt_christos.pkl' 
with open(path,'rb') as f:
    dat=pickle.load(f)
print(dat.keys())
cfgs=dat['confs']
print(cfgs)
dat['twop_mom0'].shape
# for ele in dat['twop_mom0'][:,:4]:
#     print(ele-[2.9940586e-10, 6.6129408e-10, 4.1060580e-10, 2.6694152e-10])

dict_keys(['confs', 'twop_mom0', 'twop_mom1', 'A0_mom1'])
{'0258_r0', '0186_r1', '0004_r2', '0188_r3', '0122_r3', '0090_r1', '0124_r0', '0080_r1', '0116_r2', '0036_r3', '0082_r0', '0220_r3', '0214_r0', '0104_r2', '0172_r1', '0048_r0', '0136_r0', '0176_r1', '0232_r0', '0024_r2', '0188_r1', '0090_r0', '0174_r3', '0240_r0', '0064_r2', '0088_r1', '0310_r0', '0096_r0', '0066_r3', '0148_r1', '0200_r2', '0064_r1', '0016_r2', '0194_r3', '0156_r1', '0024_r3', '0052_r3', '0000_r1', '0112_r0', '0200_r1', '0132_r0', '0138_r3', '0160_r0', '0064_r3', '0104_r3', '0104_r1', '0284_r0', '0108_r1', '0210_r0', '0218_r2', '0112_r1', '0004_r1', '0150_r0', '0098_r0', '0274_r0', '0000_r0', '0034_r0', '0154_r1', '0128_r1', '0118_r2', '0054_r3', '0086_r2', '0334_r0', '0204_r2', '0120_r2', '0144_r1', '0186_r3', '0190_r2', '0098_r1', '0010_r0', '0028_r1', '0028_r3', '0180_r2', '0314_r0', '0074_r2', '0142_r0', '0178_r3', '0140_r1', '0050_r1', '0080_r2', '0206_r3', '0134_r1', '0128_r0', '0142_r1', '0022_r2', '0154_

(496, 96)